In [ ]:

import numpy as np # Importa la librería NumPy para operaciones numéricas
from google.colab import files # Importa la función files de google.colab para cargar archivos

# ===========================
# 1. Cargar el dataset desde archivo CSV
# ===========================
uploaded = files.upload() # Abre una ventana para subir archivos y guarda el resultado
filename = list(uploaded.keys())[0] # Obtiene el nombre del primer archivo subido

# Leer líneas del archivo
with open(filename, 'r') as f: # Abre el archivo subido en modo lectura
    lines = f.readlines() # Lee todas las líneas del archivo en una lista

# Leer encabezado y determinar qué columnas excluir
header = lines[0].strip().split(';') # Lee la primera línea (encabezado), elimina espacios extra y divide por ';'
col_excluir = ["free sulfur dioxide", "density"] # Define una lista de nombres de columnas a excluir

# Índices de columnas a excluir
# Crea una lista de índices de las columnas cuyo nombre (ignorando mayúsculas y espacios)
# está en la lista col_excluir
idx_excluir = [i for i, col in enumerate(header) if col.lower().strip() in [c.lower() for c in col_excluir]]

# Índices a mantener (todas excepto las excluidas y la última)
# Crea una lista de índices para las columnas a incluir, excluyendo las de idx_excluir
# y la última columna (que es el target)
idx_incluir = [i for i in range(len(header)) if i not in idx_excluir and i != len(header)-1]
idx_target = len(header) - 1  # El índice de la última columna (la variable objetivo 'quality')

# Procesar filas válidas
data_limpia = [] # Inicializa una lista vacía para almacenar los datos limpios
for line in lines[1:]: # Itera sobre cada línea del archivo, empezando desde la segunda línea (después del encabezado)
    fila = line.strip().split(';') # Elimina espacios extra de la línea y la divide por ';'

    # Verifica si la fila tiene el número correcto de columnas y si ninguna celda está vacía
    if len(fila) == len(header) and all(cell.strip() != '' for cell in fila):
        try:
            # Intenta convertir todos los valores de la fila a float
            fila_float = [float(cell) for cell in fila]
            data_limpia.append(fila_float) # Si la conversión es exitosa, añade la fila a data_limpia
        except ValueError:
            continue # Si hay un error de conversión (no es un número), ignora la fila

# Convertir a NumPy
data = np.array(data_limpia, dtype=np.float32) # Convierte la lista de listas data_limpia a un array de NumPy con tipo float32

# ===========================
# 2. Separar características (X) y variable objetivo (y)
# ===========================
X = data[:, idx_incluir] # Selecciona todas las filas y las columnas especificadas en idx_incluir para las características (X)
y = data[:, idx_target].astype(int) # Selecciona todas las filas y la columna del target, y la convierte a tipo entero

print("Shape de X:", X.shape) # Imprime la forma (dimensiones) del array X
print("Columnas utilizadas (sin las excluidas):", [header[i] for i in idx_incluir]) # Imprime los nombres de las columnas usadas como características
print("Primeras 5 muestras de X:\n", X[:5]) # Imprime las primeras 5 filas del array X
print("Valores únicos en y:\n", np.unique(y)) # Imprime los valores únicos presentes en la variable objetivo y

# ===========================
# 3. Normalización (Z-score)
# ===========================
mean = np.mean(X, axis=0) # Calcula la media de cada columna en X
std = np.std(X, axis=0) # Calcula la desviación estándar de cada columna en X
X_normalized = (X - mean) / std # Aplica la normalización Z-score: (valor - media) / desviación estándar

# ===========================
# 4. Codificación One-Hot para salida
# ===========================
clases_unicas = np.sort(np.unique(y)) # Obtiene los valores únicos en y y los ordena
num_clases = len(clases_unicas) # Cuenta el número de clases únicas
# Crea un diccionario que mapea cada valor de clase único a un índice (0, 1, 2, ...)
class_to_index = {label: idx for idx, label in enumerate(clases_unicas)}
y_index = np.array([class_to_index[val] for val in y]) # Convierte cada valor de y a su índice correspondiente usando el diccionario
y_onehot = np.zeros((len(y_index), num_clases)) # Crea una matriz de ceros con tantas filas como muestras y tantas columnas como clases
y_onehot[np.arange(len(y_index)), y_index] = 1 # Realiza la codificación One-Hot: pone un 1 en la posición del índice de la clase para cada muestra

print("Ejemplo de salida (y) original:\n", y[:5]) # Imprime las primeras 5 muestras de la variable objetivo original
print("One-hot encoded:\n", y_onehot[:5]) # Imprime las primeras 5 muestras de la variable objetivo después de la codificación One-Hot

# ===========================
# 5. Mezclar y dividir en entrenamiento y prueba
# ===========================
np.random.seed(42)  # Establece una semilla para el generador de números aleatorios para reproducibilidad
indices = np.arange(len(X_normalized)) # Crea un array de índices desde 0 hasta el número de muestras - 1
np.random.shuffle(indices)  # Mezcla aleatoriamente los índices
X_shuffled = X_normalized[indices] # Reordena las filas de X_normalized según los índices mezclados
y_shuffled = y_onehot[indices] # Reordena las filas de y_onehot según los mismos índices mezclados

# 60% entrenamiento
train_size = int(0.6 * len(X_shuffled)) # Calcula el tamaño del conjunto de entrenamiento (60%)
X_train = X_shuffled[:train_size] # Selecciona las primeras 'train_size' filas para el conjunto de entrenamiento X
y_train = y_shuffled[:train_size] # Selecciona las primeras 'train_size' filas para el conjunto de entrenamiento y
X_test = X_shuffled[train_size:] # Selecciona las filas restantes para el conjunto de prueba X
y_test = y_shuffled[train_size:] # Selecciona las filas restantes para el conjunto de prueba y

# ===========================
# Resumen final
# ===========================
print("Tamaño entrenamiento:", X_train.shape, y_train.shape) # Imprime las dimensiones de los conjuntos de entrenamiento
print("Tamaño validación/prueba:", X_test.shape, y_test.shape) # Imprime las dimensiones de los conjuntos de prueba

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# === Funciones de activación ===
# Implementa la función de activación ReLU (Rectified Linear Unit)
def relu(x):
    return np.maximum(0, x) # Devuelve el máximo entre 0 y el valor de entrada (si es negativo, se vuelve 0)

# Implementa la derivada de la función ReLU, necesaria para la retropropagación
def relu_derivative(x):
    return (x > 0).astype(float) # Devuelve 1.0 si la entrada es mayor que 0, y 0.0 si es menor o igual a 0

# Implementa la función de activación Softmax, utilizada en la capa de salida para problemas de clasificación multiclase
def softmax(x):
    # Resta el máximo de cada fila para mejorar la estabilidad numérica y evitar la explosión de valores
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    # Normaliza los valores exponenciados para que sumen 1 a lo largo de cada fila (probabilidades)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# === Inicialización de pesos y sesgos ===
# Define el tamaño de la capa de entrada (número de características)
input_size = X_train.shape[1]         # Ej: 11 si quitaste columnas
# Define el tamaño de la primera capa oculta
hidden1_size = 16
# Define el tamaño de la segunda capa oculta
hidden2_size = 32
# Define el tamaño de la capa de salida (número de clases)
output_size = y_train.shape[1]        # 6 clases (one-hot)

# Establece una semilla para el generador de números aleatorios para asegurar la reproducibilidad de la inicialización
np.random.seed(1)
# Inicializa los pesos de la primera capa con valores aleatorios pequeños para evitar gradientes explosivos/desvanecientes
W1 = np.random.randn(input_size, hidden1_size) * 0.1
# Inicializa los sesgos de la primera capa con ceros
b1 = np.zeros((1, hidden1_size))

# Inicializa los pesos de la segunda capa oculta
W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
# Inicializa los sesgos de la segunda capa oculta
b2 = np.zeros((1, hidden2_size))

# Inicializa los pesos de la capa de salida
W3 = np.random.randn(hidden2_size, output_size) * 0.1
# Inicializa los sesgos de la capa de salida
b3 = np.zeros((1, output_size))

# === Hiperparámetros ===
# Define el número de épocas (iteraciones completas sobre todo el conjunto de entrenamiento)
epochs = 8000
# Define la tasa de aprendizaje, que controla el tamaño de los pasos durante la actualización de pesos
learning_rate = 0.1

# === Historial ===
# Lista para almacenar el valor de la función de pérdida en cada época
loss_history = []
# Lista para almacenar la precisión (accuracy) en el conjunto de entrenamiento en cada época
accuracy_history = []

# === Entrenamiento ===
# Bucle principal de entrenamiento que itera a través del número de épocas
for epoch in range(epochs):
    # --- FORWARD PASS ---
    # Calcula la entrada a la primera capa oculta (multiplicación matricial de X_train y W1, más el sesgo b1)
    Z1 = X_train @ W1 + b1
    # Aplica la función de activación ReLU a la salida de la primera capa oculta
    A1 = relu(Z1)

    # Calcula la entrada a la segunda capa oculta
    Z2 = A1 @ W2 + b2
    # Aplica la función de activación ReLU a la salida de la segunda capa oculta
    A2 = relu(Z2)

    # Calcula la entrada a la capa de salida
    Z3 = A2 @ W3 + b3
    # Aplica la función de activación Softmax a la salida de la capa de salida para obtener probabilidades
    A3 = softmax(Z3)

    # --- FUNCIÓN DE PÉRDIDA (Cross-Entropy) ---
    # Calcula la pérdida de entropía cruzada, común para problemas de clasificación multiclase
    # np.sum(y_train * np.log(A3 + 1e-8), axis=1) calcula la pérdida para cada muestra
    # -np.mean(...) calcula el promedio de la pérdida sobre todas las muestras del lote de entrenamiento
    loss = -np.mean(np.sum(y_train * np.log(A3 + 1e-8), axis=1)) # Se añade un pequeño valor (1e-8) para evitar logaritmo de cero
    # Almacena el valor de la pérdida en el historial
    loss_history.append(loss)

    # --- ACCURACY ---
    # Obtiene las clases predichas tomando el índice del valor máximo en la salida de Softmax (A3)
    y_pred_train = np.argmax(A3, axis=1)
    # Obtiene las clases verdaderas tomando el índice del valor máximo en la codificación One-Hot (y_train)
    y_true_train = np.argmax(y_train, axis=1)
    # Calcula la precisión comparando las clases predichas con las verdaderas
    acc_train = np.mean(y_pred_train == y_true_train)
    # Almacena la precisión en el historial
    accuracy_history.append(acc_train)

    # --- BACKPROPAGATION ---
    # Calcula el gradiente de la pérdida con respecto a la entrada de la capa de salida (Z3)
    dZ3 = A3 - y_train
    # Calcula el gradiente de la pérdida con respecto a los pesos de la capa de salida (W3)
    dW3 = A2.T @ dZ3
    # Calcula el gradiente de la pérdida con respecto a los sesgos de la capa de salida (b3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    # Calcula el gradiente de la pérdida con respecto a la salida de la segunda capa oculta (A2)
    dA2 = dZ3 @ W3.T
    # Calcula el gradiente de la pérdida con respecto a la entrada de la segunda capa oculta (Z2), aplicando la derivada de ReLU
    dZ2 = dA2 * relu_derivative(Z2)
    # Calcula el gradiente de la pérdida con respecto a los pesos de la segunda capa oculta (W2)
    dW2 = A1.T @ dZ2
    # Calcula el gradiente de la pérdida con respecto a los sesgos de la segunda capa oculta (b2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    # Calcula el gradiente de la pérdida con respecto a la salida de la primera capa oculta (A1)
    dA1 = dZ2 @ W2.T
    # Calcula el gradiente de la pérdida con respecto a la entrada de la primera capa oculta (Z1), aplicando la derivada de ReLU
    dZ1 = dA1 * relu_derivative(Z1)
    # Calcula el gradiente de la pérdida con respecto a los pesos de la primera capa oculta (W1)
    dW1 = X_train.T @ dZ1
    # Calcula el gradiente de la pérdida con respecto a los sesgos de la primera capa oculta (b1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    # --- ACTUALIZACIÓN DE PESOS ---
    # Actualiza los pesos y sesgos de la capa de salida utilizando el descenso de gradiente
    # Se divide por el número de muestras (X_train.shape[0]) para obtener el gradiente promedio del lote
    W3 -= learning_rate * dW3 / X_train.shape[0]
    b3 -= learning_rate * db3 / X_train.shape[0]
    # Actualiza los pesos y sesgos de la segunda capa oculta
    W2 -= learning_rate * dW2 / X_train.shape[0]
    b2 -= learning_rate * db2 / X_train.shape[0]
    # Actualiza los pesos y sesgos de la primera capa oculta
    W1 -= learning_rate * dW1 / X_train.shape[0]
    b1 -= learning_rate * db1 / X_train.shape[0]

    # --- Ajuste de Learning Rate ---
    if epoch > 5000 and epoch <= 6500:
        learning_rate = 0.05




    # --- Mostrar progreso ---
    # Imprime el progreso del entrenamiento (época, pérdida y precisión) cada 100 épocas
    if epoch % 100 == 0:
        print(f"Época {epoch} | Pérdida: {loss:.4f} | Precisión: {acc_train * 100:.2f}%")

Época 0 | Pérdida: 1.7926 | Precisión: 19.22%
Época 100 | Pérdida: 1.4505 | Precisión: 35.54%
Época 200 | Pérdida: 1.1668 | Precisión: 52.74%
Época 300 | Pérdida: 1.0359 | Precisión: 56.12%
Época 400 | Pérdida: 0.9537 | Precisión: 60.81%
Época 500 | Pérdida: 0.9070 | Precisión: 62.73%
Época 600 | Pérdida: 0.8705 | Precisión: 65.07%
Época 700 | Pérdida: 0.8398 | Precisión: 67.60%
Época 800 | Pérdida: 0.8126 | Precisión: 69.06%
Época 900 | Pérdida: 0.7868 | Precisión: 69.10%
Época 1000 | Pérdida: 0.7642 | Precisión: 69.34%
Época 1100 | Pérdida: 0.7440 | Precisión: 69.29%
Época 1200 | Pérdida: 0.7264 | Precisión: 70.56%
Época 1300 | Pérdida: 0.7117 | Precisión: 70.70%
Época 1400 | Pérdida: 0.6985 | Precisión: 71.17%
Época 1500 | Pérdida: 0.6868 | Precisión: 71.45%
Época 1600 | Pérdida: 0.6758 | Precisión: 71.82%
Época 1700 | Pérdida: 0.6650 | Precisión: 72.76%
Época 1800 | Pérdida: 0.6542 | Precisión: 72.86%
Época 1900 | Pérdida: 0.6437 | Precisión: 73.79%
Época 2000 | Pérdida: 0.6334 | P

In [ ]:
# === EVALUACIÓN ===

# 1. Hacer forward pass con los datos de prueba
# Calcula la entrada a la primera capa oculta usando los datos de prueba (X_test)
Z1_test = X_test @ W1 + b1
# Aplica la función de activación ReLU a la salida de la primera capa oculta en el conjunto de prueba
A1_test = relu(Z1_test)

# Calcula la entrada a la segunda capa oculta usando la salida de la capa anterior (A1_test)
Z2_test = A1_test @ W2 + b2
# Aplica la función de activación ReLU a la salida de la segunda capa oculta en el conjunto de prueba
A2_test = relu(Z2_test)

# Calcula la entrada a la capa de salida usando la salida de la capa anterior (A2_test)
Z3_test = A2_test @ W3 + b3
# Aplica la función de activación Softmax a la salida de la capa de salida para obtener probabilidades de predicción
A3_test = softmax(Z3_test)

# 2. Obtener clases predichas y verdaderas
# Obtiene las clases predichas tomando el índice del valor máximo en la salida de Softmax (A3_test) para cada muestra
y_pred = np.argmax(A3_test, axis=1)
# Obtiene las clases verdaderas tomando el índice del valor máximo en la codificación One-Hot (y_test) para cada muestra
y_true = np.argmax(y_test, axis=1)

# 3. Calcular accuracy
# Calcula la precisión comparando las clases predichas (y_pred) con las clases verdaderas (y_true)
accuracy = np.mean(y_pred == y_true)
# Imprime la precisión del modelo en el conjunto de prueba, formateada a dos decimales
print(f"\n🔍 Precisión (accuracy) del modelo: {accuracy * 100:.2f}%")

# 4. Matriz de confusión
conf_matrix = np.zeros((6, 6), dtype=int)  # Inicializa una matriz de ceros de tamaño 6x6 (para 6 clases) con tipo entero

# Itera sobre cada muestra en el conjunto de prueba
for i in range(len(y_true)):
    true_class = y_true[i] # Obtiene la clase verdadera de la muestra actual
    pred_class = y_pred[i] # Obtiene la clase predicha de la muestra actual
    conf_matrix[true_class, pred_class] += 1 # Incrementa el contador en la matriz de confusión en la posición (clase_verdadera, clase_predicha)

# Imprime la matriz de confusión con una etiqueta explicativa
print("\n Matriz de Confusión (filas: real, columnas: predicho):")
print(conf_matrix) # Imprime la matriz de confusión calculada

# 5. Mostrar algunas predicciones
print("\nEjemplos de predicción:")
# Itera sobre las primeras 10 muestras del conjunto de prueba
for i in range(10):
    real = y_true[i] # Obtiene la clase verdadera (índice)
    pred = y_pred[i] # Obtiene la clase predicha (índice)
    # Imprime la clase verdadera y predicha para cada ejemplo, sumando 3 para mostrar las clases originales (3 a 8)
    print(f"Vino {i+1}: Real = {real + 3}, Predicho = {pred + 3}")  # sumamos 3 para volver a clases reales


🔍 Precisión (accuracy) del modelo: 81.22%

 Matriz de Confusión (filas: real, columnas: predicho):
[[277   0   0   0   0   0]
 [  0 266   3   3   0   0]
 [  6  17 177  63   8   0]
 [  0  13  70 129  23  12]
 [  1   3   6  30  21   9]
 [  0   0   0   0   0 285]]

Ejemplos de predicción:
Vino 1: Real = 8, Predicho = 8
Vino 2: Real = 3, Predicho = 3
Vino 3: Real = 4, Predicho = 4
Vino 4: Real = 3, Predicho = 3
Vino 5: Real = 8, Predicho = 8
Vino 6: Real = 8, Predicho = 8
Vino 7: Real = 3, Predicho = 3
Vino 8: Real = 8, Predicho = 8
Vino 9: Real = 6, Predicho = 6
Vino 10: Real = 5, Predicho = 5
